# ML Model Optimization: Correlation-Guided Interaction Engineering
## Model B — Lion (claude-churro-v10-p)

**Expert**: Enzo Rodriguez | **Task ID**: TASK_11251 | **Date**: 2026-02-10

---

### What this notebook does

This notebook replicates and **enhances** the R tidymodels/broom workflow in Python.
The core idea: the model alone reaches a local equilibrium — the human element
(guided by statistical evidence) decides which interaction terms to reintroduce.

**Improvements over Model A baseline:**
- Pearson correlations with **p-values** (statistical significance gating)
- **Spearman** correlation to capture monotone non-linear relationships
- **Mutual information** to catch any non-linear feature–target signal
- **VIF** (Variance Inflation Factor) for multicollinearity — more rigorous than pairwise thresholds
- Multiple interaction types scored simultaneously (multiplicative, ratio, polynomial)
- **Ridge-regularized** model on enhanced feature set (prevents overfitting from added terms)
- **Statsmodels OLS** tidy output: p-values, confidence intervals, F-statistic (broom-style)
- Explicit **train/test gap monitoring** at every step
- Marked **HUMAN DECISION CHECKPOINTs** where the analyst reviews evidence before proceeding

## 0. Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from scipy import stats
from scipy.stats import spearmanr, pearsonr

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import (
    train_test_split, cross_val_score, cross_validate, learning_curve
)
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
pd.set_option('display.float_format', '{:.4f}'.format)

RANDOM_STATE = 42
print('Setup complete.')

## 1. Data Loading & Profile

Using the **California Housing** dataset (sklearn). This replicates a real-world
regression study with 8 numeric features and a continuous target (median house value).

| Feature | Description |
|---|---|
| MedInc | Median income in block group |
| HouseAge | Median house age |
| AveRooms | Average number of rooms |
| AveBedrms | Average number of bedrooms |
| Population | Block group population |
| AveOccup | Average household occupancy |
| Latitude | Block group latitude |
| Longitude | Block group longitude |
| **MedHouseVal** | **Target: Median house value (\$100k)** |

In [ ]:
housing = fetch_california_housing(as_frame=True)
df_raw = housing.frame.copy()
TARGET = 'MedHouseVal'

print(f'Shape: {df_raw.shape}')
print(f'Target: {TARGET}')
df_raw.describe().T.style.background_gradient(cmap='Blues', subset=['mean', 'std'])

In [ ]:
# Missing values and data types
profile = pd.DataFrame({
    'dtype': df_raw.dtypes,
    'missing': df_raw.isnull().sum(),
    'missing_pct': (df_raw.isnull().sum() / len(df_raw) * 100).round(2),
    'unique': df_raw.nunique(),
    'skewness': df_raw.skew().round(3)
})
print('Data Profile:')
display(profile)
print(f'\nNo missing values: {df_raw.isnull().sum().sum() == 0}')

In [ ]:
# IQR outlier removal on features (not target)
FEATURES = [c for c in df_raw.columns if c != TARGET]
df = df_raw.copy()

outlier_flags = pd.Series(False, index=df.index)
for col in FEATURES:
    Q1, Q3 = df[col].quantile(0.25), df[col].quantile(0.75)
    IQR = Q3 - Q1
    mask = (df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)
    outlier_flags |= mask

removed = outlier_flags.sum()
df = df[~outlier_flags].reset_index(drop=True)
print(f'Removed {removed} outlier rows ({removed/len(df_raw)*100:.1f}%)')
print(f'Cleaned dataset: {df.shape[0]} rows x {df.shape[1]} columns')

In [ ]:
# Feature distributions
fig, axes = plt.subplots(3, 3, figsize=(14, 10))
axes = axes.flatten()
for i, col in enumerate(df.columns):
    axes[i].hist(df[col], bins=40, edgecolor='black', alpha=0.7)
    axes[i].set_title(col, fontsize=11, fontweight='bold')
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Count')
    skew_val = df[col].skew()
    axes[i].text(0.97, 0.95, f'skew={skew_val:.2f}',
                 transform=axes[i].transAxes, ha='right', va='top', fontsize=9)
plt.suptitle('Feature Distributions (after outlier removal)', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('../results/feature_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

## 2. Enhanced Correlation Analysis

**Model A used only Pearson correlation values.** 
Here we compute three complementary measures and gate on statistical significance:

| Method | What it captures | When to prefer |
|---|---|---|
| Pearson + p-value | Linear relationships | Normal / symmetric data |
| Spearman | Monotone non-linear | Skewed / ordinal data |
| Mutual Information | Any non-linear signal | Always useful as a complement |

In [ ]:
# Pearson correlation with p-values (feature vs target)
pearson_rows = []
for feat in FEATURES:
    r, p = pearsonr(df[feat], df[TARGET])
    pearson_rows.append({'feature': feat, 'pearson_r': r, 'pearson_abs': abs(r), 'pearson_p': p})

pearson_df = pd.DataFrame(pearson_rows).sort_values('pearson_abs', ascending=False)
pearson_df['significant_0.05'] = pearson_df['pearson_p'] < 0.05
print('Pearson correlation with target (sorted by |r|):')
display(pearson_df.set_index('feature'))

In [ ]:
# Spearman correlation with p-values
spearman_rows = []
for feat in FEATURES:
    rho, p = spearmanr(df[feat], df[TARGET])
    spearman_rows.append({'feature': feat, 'spearman_rho': rho, 'spearman_abs': abs(rho), 'spearman_p': p})

spearman_df = pd.DataFrame(spearman_rows).sort_values('spearman_abs', ascending=False)

# Mutual information
X_for_mi = df[FEATURES].values
y_for_mi = df[TARGET].values
mi_scores = mutual_info_regression(X_for_mi, y_for_mi, random_state=RANDOM_STATE)
mi_df = pd.DataFrame({'feature': FEATURES, 'mutual_info': mi_scores}).sort_values('mutual_info', ascending=False)

# Merge all three into one summary table
corr_summary = (
    pearson_df[['feature', 'pearson_r', 'pearson_abs', 'pearson_p']]
    .merge(spearman_df[['feature', 'spearman_rho', 'spearman_abs']], on='feature')
    .merge(mi_df, on='feature')
    .sort_values('mutual_info', ascending=False)
    .reset_index(drop=True)
)
corr_summary['rank_pearson'] = corr_summary['pearson_abs'].rank(ascending=False).astype(int)
corr_summary['rank_spearman'] = corr_summary['spearman_abs'].rank(ascending=False).astype(int)
corr_summary['rank_mi'] = corr_summary['mutual_info'].rank(ascending=False).astype(int)

print('Unified correlation summary (all three methods):')
display(corr_summary.set_index('feature')[
    ['pearson_r', 'pearson_p', 'spearman_rho', 'mutual_info',
     'rank_pearson', 'rank_spearman', 'rank_mi']
])

In [ ]:
# Side-by-side bar chart comparing all three methods
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

def bar_corr(ax, series, labels, title, color):
    sorted_idx = np.argsort(np.abs(series))[::-1]
    vals = np.array(series)[sorted_idx]
    labs = np.array(labels)[sorted_idx]
    colors = ['#e74c3c' if v < 0 else color for v in vals]
    ax.barh(labs, vals, color=colors, alpha=0.8, edgecolor='k', linewidth=0.4)
    ax.axvline(0, color='black', linewidth=0.8)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xlabel('Score')
    ax.grid(axis='x', alpha=0.3)

bar_corr(axes[0], corr_summary['pearson_r'].values,
         corr_summary['feature'].values, 'Pearson r', '#3498db')
bar_corr(axes[1], corr_summary['spearman_rho'].values,
         corr_summary['feature'].values, 'Spearman rho', '#2ecc71')
bar_corr(axes[2], corr_summary['mutual_info'].values,
         corr_summary['feature'].values, 'Mutual Information', '#9b59b6')

plt.suptitle(f'Feature Correlations with {TARGET}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/correlation_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Full pairwise correlation heatmap with annotations
corr_matrix = df[FEATURES].corr(method='pearson')

fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(
    corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
    center=0, square=True, linewidths=0.5, ax=ax,
    cbar_kws={'shrink': 0.8}
)
ax.set_title('Feature-Feature Pearson Correlation (lower triangle)', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/correlation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Variance Inflation Factor (VIF) — detects multicollinearity more rigorously than pairwise thresholds
# VIF > 10 is a strong multicollinearity signal; VIF > 5 is moderate
X_vif = df[FEATURES].copy()
X_vif_scaled = StandardScaler().fit_transform(X_vif)
X_vif_sm = pd.DataFrame(X_vif_scaled, columns=FEATURES)

vif_data = pd.DataFrame()
vif_data['feature'] = FEATURES
vif_data['VIF'] = [variance_inflation_factor(X_vif_sm.values, i) for i in range(len(FEATURES))]
vif_data = vif_data.sort_values('VIF', ascending=False).reset_index(drop=True)
vif_data['multicollinearity'] = pd.cut(
    vif_data['VIF'], bins=[0, 5, 10, np.inf], labels=['Low', 'Moderate', 'High']
)

print('Variance Inflation Factor (VIF):')
print('  VIF < 5   = Low multicollinearity (safe to include)')
print('  VIF 5-10  = Moderate — monitor closely')
print('  VIF > 10  = High — consider removing or combining\n')
display(vif_data)

In [ ]:
# Interaction candidate scoring (enhanced over Model A)
# Model A score = (|r1_target| + |r2_target|) * |r12_inter_feature|
# Model B adds mutual information weight: gives extra credit when MI is high

# Gate on significance: only use features with p < 0.05
sig_features = pearson_df[pearson_df['pearson_p'] < 0.05]['feature'].tolist()
print(f'Features with significant Pearson correlation (p < 0.05): {sig_features}\n')

# Build lookup dicts
pearson_lookup = dict(zip(pearson_df['feature'], pearson_df['pearson_abs']))
mi_lookup = dict(zip(mi_df['feature'], mi_df['mutual_info']))

candidates = []
for i, f1 in enumerate(sig_features):
    for f2 in sig_features[i+1:]:
        inter_corr = abs(corr_matrix.loc[f1, f2])
        # Include pairs with moderate inter-feature correlation (not too low, not too high)
        if 0.05 <= inter_corr <= 0.80:
            r1 = pearson_lookup.get(f1, 0)
            r2 = pearson_lookup.get(f2, 0)
            mi1 = mi_lookup.get(f1, 0)
            mi2 = mi_lookup.get(f2, 0)
            # Enhanced score: linear + MI components
            linear_score = (r1 + r2) * inter_corr
            mi_score = (mi1 + mi2) * inter_corr
            combined_score = 0.5 * linear_score + 0.5 * mi_score
            candidates.append({
                'feature_1': f1, 'feature_2': f2,
                'f1_pearson': round(r1, 4), 'f2_pearson': round(r2, 4),
                'f1_MI': round(mi1, 4), 'f2_MI': round(mi2, 4),
                'inter_feature_corr': round(inter_corr, 4),
                'linear_score': round(linear_score, 4),
                'mi_score': round(mi_score, 4),
                'combined_score': round(combined_score, 4)
            })

candidates_df = pd.DataFrame(candidates).sort_values('combined_score', ascending=False).reset_index(drop=True)
print(f'Total interaction candidates: {len(candidates_df)}')
print('\nTop 15 Interaction Candidates (Model B enhanced scoring):')
display(candidates_df.head(15))

---
## HUMAN DECISION CHECKPOINT 1: Select Interaction Candidates

**Review the table above before proceeding.**

Ask yourself:
1. Do any of the top pairs have domain-level justification?
   - `MedInc × AveRooms` — wealthier blocks tend to have more spacious homes; the *combination* may better explain value
   - `Latitude × Longitude` — geographic location as a 2D interaction captures region (e.g., coastal vs inland)
   - `MedInc × HouseAge` — new homes in rich areas vs. old homes in rich areas behave differently
   - `Population × AveOccup` — density effects
2. Does the `inter_feature_corr` suggest the two features are **not** redundant (VIF risk)?
3. Is the `mi_score` consistent with the `linear_score`? A large gap suggests non-linearity worth capturing.

**The cell below lets you configure which interactions to carry forward.**

In [ ]:
# ── ANALYST CONFIGURATION ──────────────────────────────────────────────
# Edit TOP_N_AUTO to automatically take the top N by combined_score,
# OR override by listing specific pairs in MANUAL_PAIRS.
# Set MANUAL_PAIRS = [] to use automatic selection only.
TOP_N_AUTO = 12
MANUAL_PAIRS = []   # e.g. [('MedInc', 'Latitude'), ('HouseAge', 'AveRooms')]
# ───────────────────────────────────────────────────────────────────────

auto_pairs = [
    (row['feature_1'], row['feature_2'])
    for _, row in candidates_df.head(TOP_N_AUTO).iterrows()
]
all_pairs = list(dict.fromkeys(auto_pairs + MANUAL_PAIRS))  # preserve order, deduplicate

print(f'Proceeding with {len(all_pairs)} interaction pairs:')
for p in all_pairs:
    print(f'  {p[0]} x {p[1]}')

## 3. Interaction Engineering

We create **three interaction types** for each selected pair, then let cross-validation
decide which form actually helps. This avoids assuming multiplicative is always best.

In [ ]:
def make_interactions(df, pairs):
    """Create multiplicative, ratio, and difference interactions for all pairs."""
    terms = {}
    eps = 1e-8
    for f1, f2 in pairs:
        terms[f'{f1}_x_{f2}']       = df[f1] * df[f2]            # multiplicative
        terms[f'{f1}_div_{f2}']     = df[f1] / (df[f2] + eps)    # ratio
        terms[f'{f1}_minus_{f2}']   = df[f1] - df[f2]            # difference
    # Polynomial terms for top-MI features
    top_mi_features = mi_df['feature'].head(3).tolist()
    for feat in top_mi_features:
        terms[f'{feat}_sq'] = df[feat] ** 2
    return pd.DataFrame(terms, index=df.index)

interactions_df = make_interactions(df, all_pairs)
print(f'Created {interactions_df.shape[1]} raw interaction terms:')
for col in interactions_df.columns:
    print(f'  {col}')

In [ ]:
# Evaluate each interaction term via 5-fold CV
# Metric: improvement in R2 over the baseline (no interactions)
X_base = df[FEATURES].copy()
y = df[TARGET].copy()

scaler = StandardScaler()
X_base_scaled = scaler.fit_transform(X_base)

rf_eval = RandomForestRegressor(n_estimators=50, random_state=RANDOM_STATE, n_jobs=-1)
baseline_cv = cross_val_score(rf_eval, X_base_scaled, y, cv=5, scoring='r2')
baseline_mean = baseline_cv.mean()
print(f'Baseline CV R2: {baseline_mean:.4f} (std {baseline_cv.std():.4f})')

importance_rows = []
for col in interactions_df.columns:
    X_with = np.hstack([X_base_scaled, interactions_df[[col]].values])
    scores = cross_val_score(rf_eval, X_with, y, cv=5, scoring='r2')
    improvement = scores.mean() - baseline_mean
    importance_rows.append({
        'term': col,
        'cv_r2': round(scores.mean(), 5),
        'cv_std': round(scores.std(), 5),
        'improvement': round(improvement, 5),
        'improvement_pct': round(improvement / abs(baseline_mean) * 100, 3)
    })

importance_result = pd.DataFrame(importance_rows).sort_values('improvement', ascending=False).reset_index(drop=True)
print(f'\nInteraction Term Importance (sorted by R2 improvement):')
display(importance_result)

In [ ]:
# Visualize interaction importance
fig, ax = plt.subplots(figsize=(12, max(6, len(importance_result) * 0.35)))
colors = ['#27ae60' if v >= 0 else '#e74c3c' for v in importance_result['improvement']]
ax.barh(importance_result['term'][::-1], importance_result['improvement'][::-1],
        color=colors[::-1], alpha=0.85, edgecolor='k', linewidth=0.4)
ax.axvline(0, color='black', linewidth=1)
ax.set_xlabel('R² Improvement over Baseline', fontsize=12)
ax.set_title('Interaction Term R² Improvement (5-fold CV)', fontsize=13, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('../results/interaction_importance.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Select interactions with positive improvement
# Threshold: improvement must be > 0 (strictly beneficial)
selected_terms = importance_result[importance_result['improvement'] > 0]['term'].tolist()
print(f'Interactions with positive R2 improvement: {len(selected_terms)}')
print(f'Interactions that hurt performance: {(importance_result["improvement"] < 0).sum()}')
print(f'\nSelected terms: {selected_terms}')

# Build enhanced dataset: original features + selected interactions
df_enhanced = pd.concat([df[FEATURES + [TARGET]], interactions_df[selected_terms]], axis=1)
print(f'\nEnhanced dataset: {df_enhanced.shape[1] - 1} features ({len(FEATURES)} original + {len(selected_terms)} interactions)')

## 4. Overfitting Detection: Train vs. Test Gap

Before committing to the enhanced feature set, we explicitly check whether adding
interactions inflates training performance without improving generalization.

**Rule of thumb**: if Train R² >> Test R², we are overfitting — add Ridge regularization.

In [ ]:
def overfit_check(data, target, label, scaler=None, random_state=42):
    """Return train/test R2 for Random Forest and a gap indicator."""
    feats = [c for c in data.columns if c != target]
    X = data[feats].values
    y_vals = data[target].values
    X_tr, X_te, y_tr, y_te = train_test_split(X, y_vals, test_size=0.2, random_state=random_state)
    if scaler:
        sc = StandardScaler()
        X_tr = sc.fit_transform(X_tr)
        X_te = sc.transform(X_te)
    rf = RandomForestRegressor(n_estimators=100, random_state=random_state, n_jobs=-1)
    rf.fit(X_tr, y_tr)
    train_r2 = r2_score(y_tr, rf.predict(X_tr))
    test_r2  = r2_score(y_te, rf.predict(X_te))
    gap = train_r2 - test_r2
    print(f'[{label}] Train R2: {train_r2:.4f} | Test R2: {test_r2:.4f} | Gap: {gap:.4f}')
    return {'label': label, 'train_r2': train_r2, 'test_r2': test_r2, 'gap': gap}

gap_rows = []
gap_rows.append(overfit_check(df[FEATURES + [TARGET]], TARGET, 'Baseline (no interactions)'))
gap_rows.append(overfit_check(df_enhanced, TARGET, f'Enhanced (+{len(selected_terms)} interactions)'))

gap_df = pd.DataFrame(gap_rows)
print()
display(gap_df)

---
## HUMAN DECISION CHECKPOINT 2: Regularization Strategy

**Review the Train/Test Gap above before configuring models.**

| Gap size | Recommendation |
|---|---|
| < 0.02 | No overfitting — Random Forest with interactions is fine |
| 0.02 – 0.08 | Mild overfitting — add Ridge alongside RF |
| > 0.08 | Overfitting — use Ridge/Lasso primarily; reduce interaction count |

The model suite below trains Ridge and ElasticNet alongside Random Forest, so all
scenarios are covered. The comparison table will tell you which approach wins.

## 5. Model Training: Baseline vs Enhanced vs Regularized

All models are trained with the **same 80/20 split** and **5-fold cross-validation**.
Scaling is applied uniformly (fitted on train only, applied to test).

In [ ]:
def train_and_evaluate(data, target, model, label, X_tr=None, X_te=None, y_tr=None, y_te=None, cv=5):
    """Train a model, run CV, return a result dict."""
    feats = [c for c in data.columns if c != target]
    X = data[feats].values
    y_vals = data[target].values

    if X_tr is None:
        X_tr, X_te, y_tr, y_te = train_test_split(X, y_vals, test_size=0.2, random_state=RANDOM_STATE)

    sc = StandardScaler()
    X_tr_sc = sc.fit_transform(X_tr)
    X_te_sc  = sc.transform(X_te)

    cv_scores = cross_val_score(model, X_tr_sc, y_tr, cv=cv, scoring='r2')

    model.fit(X_tr_sc, y_tr)
    y_train_pred = model.predict(X_tr_sc)
    y_test_pred  = model.predict(X_te_sc)

    return {
        'label': label,
        'n_features': len(feats),
        'cv_r2_mean': round(cv_scores.mean(), 4),
        'cv_r2_std':  round(cv_scores.std(), 4),
        'train_r2':   round(r2_score(y_tr, y_train_pred), 4),
        'test_r2':    round(r2_score(y_te, y_test_pred), 4),
        'test_rmse':  round(np.sqrt(mean_squared_error(y_te, y_test_pred)), 4),
        'test_mae':   round(mean_absolute_error(y_te, y_test_pred), 4),
        'gap':        round(r2_score(y_tr, y_train_pred) - r2_score(y_te, y_test_pred), 4),
        'model_obj':  model,
        'y_test':     y_te,
        'y_pred':     y_test_pred,
        'y_train':    y_tr,
        'y_train_pred': y_train_pred
    }

results = []
data_base = df[FEATURES + [TARGET]]
data_enh  = df_enhanced

# Baseline models
results.append(train_and_evaluate(data_base, TARGET, LinearRegression(), 'Baseline — Linear Regression'))
results.append(train_and_evaluate(data_base, TARGET, Ridge(alpha=1.0),   'Baseline — Ridge'))
results.append(train_and_evaluate(data_base, TARGET, RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1), 'Baseline — Random Forest'))
results.append(train_and_evaluate(data_base, TARGET, GradientBoostingRegressor(n_estimators=100, random_state=RANDOM_STATE), 'Baseline — Gradient Boosting'))

# Enhanced models (+interactions, no regularization)
results.append(train_and_evaluate(data_enh, TARGET, LinearRegression(), 'Enhanced — Linear Regression'))
results.append(train_and_evaluate(data_enh, TARGET, Ridge(alpha=1.0),   'Enhanced — Ridge'))
results.append(train_and_evaluate(data_enh, TARGET, Ridge(alpha=10.0),  'Enhanced — Ridge (alpha=10)'))
results.append(train_and_evaluate(data_enh, TARGET, Lasso(alpha=0.01),  'Enhanced — Lasso'))
results.append(train_and_evaluate(data_enh, TARGET, ElasticNet(alpha=0.01, l1_ratio=0.5), 'Enhanced — ElasticNet'))
results.append(train_and_evaluate(data_enh, TARGET, RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1), 'Enhanced — Random Forest'))
results.append(train_and_evaluate(data_enh, TARGET, GradientBoostingRegressor(n_estimators=100, random_state=RANDOM_STATE), 'Enhanced — Gradient Boosting'))

print('Training complete.')

In [ ]:
# Model comparison table
compare_cols = ['label', 'n_features', 'cv_r2_mean', 'cv_r2_std', 'train_r2', 'test_r2', 'test_rmse', 'test_mae', 'gap']
compare_df = pd.DataFrame([{k: r[k] for k in compare_cols} for r in results])
compare_df = compare_df.sort_values('test_r2', ascending=False).reset_index(drop=True)

print('Model Comparison (sorted by Test R2):')
display(
    compare_df.style
    .background_gradient(subset=['test_r2', 'cv_r2_mean'], cmap='Greens')
    .background_gradient(subset=['gap'], cmap='Reds')
    .format({'cv_r2_mean': '{:.4f}', 'test_r2': '{:.4f}', 'gap': '{:.4f}'})
)

best = compare_df.iloc[0]
print(f'\nBest model: {best["label"]}')
print(f'  Test R2: {best["test_r2"]:.4f} | CV R2: {best["cv_r2_mean"]:.4f} | Gap: {best["gap"]:.4f}')

In [ ]:
# Visual comparison: Test R2 with Train/Test gap
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

ax = axes[0]
labels_short = [r['label'].replace('Baseline — ', 'B: ').replace('Enhanced — ', 'E: ') for r in results]
test_r2s  = [r['test_r2']  for r in results]
train_r2s = [r['train_r2'] for r in results]
x_pos = np.arange(len(results))
ax.barh(x_pos, test_r2s, alpha=0.8, color='#3498db', label='Test R2', edgecolor='k', linewidth=0.4)
ax.barh(x_pos, train_r2s, alpha=0.3, color='#e74c3c', label='Train R2', edgecolor='k', linewidth=0.3)
ax.set_yticks(x_pos)
ax.set_yticklabels(labels_short, fontsize=9)
ax.set_xlabel('R²')
ax.set_title('Train vs Test R² (all models)', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(axis='x', alpha=0.3)

ax2 = axes[1]
gaps = [r['gap'] for r in results]
gap_colors = ['#e74c3c' if g > 0.05 else '#f39c12' if g > 0.02 else '#27ae60' for g in gaps]
ax2.barh(x_pos, gaps, color=gap_colors, alpha=0.8, edgecolor='k', linewidth=0.4)
ax2.set_yticks(x_pos)
ax2.set_yticklabels(labels_short, fontsize=9)
ax2.axvline(0.02, color='orange', linestyle='--', linewidth=1, label='Mild threshold (0.02)')
ax2.axvline(0.08, color='red',    linestyle='--', linewidth=1, label='High threshold (0.08)')
ax2.set_xlabel('Train R² − Test R²')
ax2.set_title('Overfitting Gap by Model', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('../results/model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Statsmodels OLS — Broom-Style Tidy Output

Random Forest gives strong predictions but no interpretability. Here we fit
**Ordinary Least Squares** on the enhanced feature set to get:
- **Coefficients** with p-values and confidence intervals
- **F-statistic** (overall model significance)
- **AIC / BIC** (model selection criteria)

This is the Python equivalent of R's `broom::tidy()` and `broom::glance()`.

In [ ]:
# OLS on the enhanced feature set (scaled)
enh_feats = [c for c in df_enhanced.columns if c != TARGET]
X_enh = df_enhanced[enh_feats].values
y_vals = df_enhanced[TARGET].values

X_tr, X_te, y_tr, y_te = train_test_split(X_enh, y_vals, test_size=0.2, random_state=RANDOM_STATE)
sc_ols = StandardScaler()
X_tr_sc = sc_ols.fit_transform(X_tr)
X_tr_sm = sm.add_constant(X_tr_sc)   # add intercept

ols_model = sm.OLS(y_tr, X_tr_sm).fit()
print(ols_model.summary())

In [ ]:
# broom::tidy() equivalent — coefficient table
col_names_ols = ['const'] + enh_feats
ci = ols_model.conf_int()  # numpy array (n_params, 2): col 0=lower, col 1=upper
tidy_df = pd.DataFrame({
    'term':        col_names_ols,
    'estimate':    list(ols_model.params),
    'std_error':   list(ols_model.bse),
    't_statistic': list(ols_model.tvalues),
    'p_value':     list(ols_model.pvalues),
    'ci_lower':    list(ci[:, 0]),
    'ci_upper':    list(ci[:, 1])
})
tidy_df['significant'] = tidy_df['p_value'] < 0.05

print('broom::tidy() — Coefficient table:')
display(
    tidy_df.sort_values('p_value').style
    .background_gradient(subset=['p_value'], cmap='RdYlGn_r')
    .format({'estimate': '{:.4f}', 'p_value': '{:.4e}', 'ci_lower': '{:.4f}', 'ci_upper': '{:.4f}'})
)

In [ ]:
# broom::glance() equivalent — model-level statistics
glance_df = pd.DataFrame([{
    'r_squared':      round(ols_model.rsquared, 4),
    'adj_r_squared':  round(ols_model.rsquared_adj, 4),
    'f_statistic':    round(ols_model.fvalue, 2),
    'f_pvalue':       round(ols_model.f_pvalue, 6),
    'aic':            round(ols_model.aic, 2),
    'bic':            round(ols_model.bic, 2),
    'n_obs':          int(ols_model.nobs),
    'df_model':       int(ols_model.df_model),
    'df_residuals':   int(ols_model.df_resid)
}])
print('broom::glance() — Model-level statistics:')
display(glance_df)

In [ ]:
# Coefficient plot with 95% CI (exclude intercept)
coef_plot = tidy_df[tidy_df['term'] != 'const'].sort_values('estimate')

fig, ax = plt.subplots(figsize=(10, max(6, len(coef_plot) * 0.4)))
colors = ['#27ae60' if s else '#95a5a6' for s in coef_plot['significant']]
ax.barh(coef_plot['term'], coef_plot['estimate'], color=colors, alpha=0.8, edgecolor='k', linewidth=0.3)
ax.errorbar(
    coef_plot['estimate'], coef_plot['term'],
    xerr=[
        coef_plot['estimate'] - coef_plot['ci_lower'],
        coef_plot['ci_upper'] - coef_plot['estimate']
    ],
    fmt='none', color='black', capsize=3, linewidth=1
)
ax.axvline(0, color='black', linewidth=1)
ax.set_xlabel('OLS Coefficient (standardized features)', fontsize=11)
ax.set_title('OLS Coefficients with 95% CI\n(green = p < 0.05)', fontsize=12, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('../results/ols_coefficients.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Model Evaluation & Residual Diagnostics

Full diagnostic plots for the best-performing enhanced model.

In [ ]:
# Identify the best enhanced model from results list
best_result = max(
    [r for r in results if r['label'].startswith('Enhanced')],
    key=lambda r: r['test_r2']
)
print(f'Best enhanced model: {best_result["label"]}')
print(f'  Test R2:  {best_result["test_r2"]:.4f}')
print(f'  Test RMSE: {best_result["test_rmse"]:.4f}')
print(f'  Test MAE:  {best_result["test_mae"]:.4f}')
print(f'  Overfit gap: {best_result["gap"]:.4f}')

y_test  = best_result['y_test']
y_pred  = best_result['y_pred']
residuals = y_test - y_pred

In [ ]:
# 4-panel residual diagnostic (mirrors R's plot(lm(...)))
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Predicted vs Actual
ax = axes[0, 0]
ax.scatter(y_test, y_pred, alpha=0.4, edgecolors='k', linewidth=0.3, s=12)
mn, mx = min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())
ax.plot([mn, mx], [mn, mx], 'r--', lw=2, label='Perfect prediction')
r2 = r2_score(y_test, y_pred)
ax.text(0.05, 0.95, f'R² = {r2:.4f}', transform=ax.transAxes, va='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))
ax.set_xlabel('Actual'); ax.set_ylabel('Predicted')
ax.set_title('Predicted vs Actual', fontweight='bold'); ax.legend(); ax.grid(alpha=0.3)

# 2. Residuals vs Fitted
ax = axes[0, 1]
ax.scatter(y_pred, residuals, alpha=0.4, edgecolors='k', linewidth=0.3, s=12)
ax.axhline(0, color='red', lw=2, linestyle='--')
ax.set_xlabel('Fitted Values'); ax.set_ylabel('Residuals')
ax.set_title('Residuals vs Fitted', fontweight='bold'); ax.grid(alpha=0.3)

# 3. Q-Q plot
ax = axes[1, 0]
stats.probplot(residuals, dist='norm', plot=ax)
ax.set_title('Normal Q-Q Plot', fontweight='bold'); ax.grid(alpha=0.3)

# 4. Scale-Location
ax = axes[1, 1]
std_resid = residuals / residuals.std()
ax.scatter(y_pred, np.sqrt(np.abs(std_resid)), alpha=0.4, edgecolors='k', linewidth=0.3, s=12)
ax.set_xlabel('Fitted Values'); ax.set_ylabel('sqrt(|Std Residuals|)')
ax.set_title('Scale-Location', fontweight='bold'); ax.grid(alpha=0.3)

plt.suptitle(f'Residual Diagnostics — {best_result["label"]}', fontsize=13, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('../results/residual_diagnostics.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Statistical tests on residuals
shapiro_stat, shapiro_p = stats.shapiro(residuals[:5000])  # limit to 5000 for Shapiro
dw_stat = np.sum(np.diff(residuals)**2) / np.sum(residuals**2)

print('Residual Assumption Tests:')
print(f'  Shapiro-Wilk normality: stat={shapiro_stat:.4f}, p={shapiro_p:.4e}')
print(f'    Residuals are normal: {shapiro_p > 0.05}')
print(f'  Durbin-Watson: {dw_stat:.4f}')
if dw_stat < 1.5:
    print('    Positive autocorrelation detected')
elif dw_stat > 2.5:
    print('    Negative autocorrelation detected')
else:
    print('    No significant autocorrelation')

print(f'\nResidual Stats:')
print(f'  Mean:     {residuals.mean():.6f}  (should be ~0)')
print(f'  Std Dev:  {residuals.std():.4f}')
print(f'  Skewness: {stats.skew(residuals):.4f}  (should be ~0)')
print(f'  Kurtosis: {stats.kurtosis(residuals):.4f}  (should be ~0)')

## 8. Feature Importance Analysis

Which of the added interaction terms actually ended up being important to the best tree model?

In [ ]:
best_model_obj = best_result['model_obj']

if hasattr(best_model_obj, 'feature_importances_'):
    enh_feat_names = [c for c in df_enhanced.columns if c != TARGET]
    fi_df = pd.DataFrame({
        'feature':    enh_feat_names,
        'importance': best_model_obj.feature_importances_
    }).sort_values('importance', ascending=False).reset_index(drop=True)

    fi_df['is_interaction'] = fi_df['feature'].isin(selected_terms)

    print(f'Top 20 features by importance:')
    display(fi_df.head(20))

    # Pie chart: original vs interaction importance share
    orig_share  = fi_df[~fi_df['is_interaction']]['importance'].sum()
    inter_share = fi_df[ fi_df['is_interaction']]['importance'].sum()
    print(f'\nImportance share — Original features: {orig_share:.1%} | Interaction terms: {inter_share:.1%}')
else:
    print('Model does not expose feature importances — use OLS tidy output instead.')
    fi_df = None

In [ ]:
if fi_df is not None:
    fig, ax = plt.subplots(figsize=(11, 7))
    top20 = fi_df.head(20)
    colors = ['#e67e22' if inter else '#3498db' for inter in top20['is_interaction']]
    ax.barh(top20['feature'][::-1], top20['importance'][::-1],
            color=colors[::-1], alpha=0.85, edgecolor='k', linewidth=0.4)
    ax.set_xlabel('Feature Importance', fontsize=12)
    ax.set_title(f'Top 20 Feature Importances\n({best_result["label"]})', fontsize=12, fontweight='bold')
    # Legend
    from matplotlib.patches import Patch
    legend_elems = [Patch(color='#3498db', alpha=0.85, label='Original feature'),
                    Patch(color='#e67e22', alpha=0.85, label='Interaction term')]
    ax.legend(handles=legend_elems, fontsize=10)
    ax.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig('../results/feature_importance.png', dpi=150, bbox_inches='tight')
    plt.show()

## 9. Learning Curves

Learning curves show whether the model benefits from more data (underfitting)
or has already plateaued (overfitting). Comparing baseline vs enhanced curves
confirms that the added interactions generalize.

In [ ]:
def plot_learning_curve(ax, X, y, model, title, color='#3498db'):
    train_sizes, train_scores, val_scores = learning_curve(
        model, X, y, cv=4,
        train_sizes=np.linspace(0.1, 1.0, 8),
        scoring='r2', n_jobs=-1
    )
    tr_mean, tr_std = train_scores.mean(axis=1), train_scores.std(axis=1)
    va_mean, va_std = val_scores.mean(axis=1), val_scores.std(axis=1)
    ax.plot(train_sizes, tr_mean, 'o-', color=color, label='Train')
    ax.fill_between(train_sizes, tr_mean - tr_std, tr_mean + tr_std, alpha=0.15, color=color)
    ax.plot(train_sizes, va_mean, 's--', color='#e74c3c', label='Validation')
    ax.fill_between(train_sizes, va_mean - va_std, va_mean + va_std, alpha=0.15, color='#e74c3c')
    ax.set_xlabel('Training Set Size'); ax.set_ylabel('R²')
    ax.set_title(title, fontweight='bold'); ax.legend(); ax.grid(alpha=0.3)

rf_base = RandomForestRegressor(n_estimators=50, random_state=RANDOM_STATE, n_jobs=-1)
rf_enh  = RandomForestRegressor(n_estimators=50, random_state=RANDOM_STATE, n_jobs=-1)

sc_lc = StandardScaler()
X_base_lc = sc_lc.fit_transform(df[FEATURES].values)
X_enh_lc  = StandardScaler().fit_transform(df_enhanced[[c for c in df_enhanced.columns if c != TARGET]].values)
y_lc = df[TARGET].values

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
plot_learning_curve(axes[0], X_base_lc, y_lc, rf_base, 'Baseline (no interactions)', '#3498db')
plot_learning_curve(axes[1], X_enh_lc,  y_lc, rf_enh,  f'Enhanced (+{len(selected_terms)} interactions)', '#2ecc71')
plt.suptitle('Learning Curves: Random Forest', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('../results/learning_curves.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Summary & Recommendations

This cell auto-generates a summary from the results computed above.

In [ ]:
# Baseline best
best_base = max([r for r in results if r['label'].startswith('Baseline')], key=lambda r: r['test_r2'])
best_enh  = max([r for r in results if r['label'].startswith('Enhanced')], key=lambda r: r['test_r2'])

r2_lift = best_enh['test_r2'] - best_base['test_r2']
rmse_reduction = best_base['test_rmse'] - best_enh['test_rmse']

sig_interactions = tidy_df[tidy_df['significant'] & tidy_df['term'].isin(selected_terms)]['term'].tolist()

print('=' * 65)
print('  ML OPTIMIZATION SUMMARY — Model B (Lion)')
print('=' * 65)
print(f'\nDataset:  California Housing  ({df.shape[0]} samples, {len(FEATURES)} features)')
print(f'Target:   {TARGET}')

print(f'\nCorrelation Analysis:')
print(f'  Significant features (p < 0.05):  {len(sig_features)}/{len(FEATURES)}')
print(f'  Features with high VIF (>10):     {(vif_data["VIF"] > 10).sum()}')
print(f'  Interaction candidates identified: {len(candidates_df)}')

print(f'\nInteraction Engineering:')
print(f'  Pairs evaluated:   {len(all_pairs)}')
print(f'  Terms created:     {len(interactions_df.columns)}')
print(f'  Terms selected:    {len(selected_terms)} (positive CV R² improvement)')

print(f'\nModel Performance:')
print(f'  Best Baseline:  {best_base["label"]}')
print(f'    Test R2={best_base["test_r2"]:.4f}  RMSE={best_base["test_rmse"]:.4f}  Gap={best_base["gap"]:.4f}')
print(f'  Best Enhanced:  {best_enh["label"]}')
print(f'    Test R2={best_enh["test_r2"]:.4f}  RMSE={best_enh["test_rmse"]:.4f}  Gap={best_enh["gap"]:.4f}')
print(f'\n  R² lift from interactions: +{r2_lift:.4f}')
print(f'  RMSE reduction:             -{rmse_reduction:.4f}')

print(f'\nStatistically Significant Interaction Terms (OLS, p < 0.05):')
if sig_interactions:
    for t in sig_interactions:
        row = tidy_df[tidy_df['term'] == t].iloc[0]
        print(f'  {t}: coef={row["estimate"]:.4f}, p={row["p_value"]:.4e}')
else:
    print('  None (interactions may still help tree models non-linearly)')

print('\nNext Steps:')
print('  1. If gap > 0.08, reduce TOP_N_AUTO in Checkpoint 1 and re-run')
print('  2. If R2 lift < 0.005, explore ratio/polynomial interactions more aggressively')
print('  3. For production, use the Enhanced Ridge model (best regularization/interpretability balance)')
print('=' * 65)

In [ ]:
# Save key outputs
import os
os.makedirs('../results', exist_ok=True)

compare_df.drop(columns=['model_obj', 'y_test', 'y_pred', 'y_train', 'y_train_pred'], errors='ignore').to_csv('../results/model_comparison.csv', index=False)
corr_summary.to_csv('../results/correlation_summary.csv', index=False)
candidates_df.to_csv('../results/interaction_candidates.csv', index=False)
importance_result.to_csv('../results/interaction_importance.csv', index=False)
tidy_df.to_csv('../results/ols_tidy.csv', index=False)
if fi_df is not None:
    fi_df.to_csv('../results/feature_importance.csv', index=False)

print('Results saved to ../results/')
print('  model_comparison.csv')
print('  correlation_summary.csv')
print('  interaction_candidates.csv')
print('  interaction_importance.csv')
print('  ols_tidy.csv')
print('  feature_importance.csv')